<a href="https://colab.research.google.com/github/SayaliDeodikar/Project_Work/blob/main/merge_sort_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycuda 
# !apt-get install -y cuda-toolkit-11-0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661975 sha256=f2bd520ac87514983730d7de8bf4ff83314ab30aac789b35dcef77a4ee060a89
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69855 sha256=892288c2bb1d0e3bdf3f932353e08cbd904b874d5586dfdf22ff3e5f6ede7be9
  Stored in directory: /root/.cache/pip/wheels/19/0

In [3]:
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda import gpuarray
from pycuda.compiler import SourceModule 
import numpy as np
import time
import random

def merge(left, right, output):
    i, j, k = 0, 0, 0
    n_left, n_right = len(left), len(right)

    while i < n_left and j < n_right:
        if left[i] <= right[j]:
            output[k] = left[i]
            i += 1
        else:
            output[k] = right[j]
            j += 1
        k += 1

    while i < n_left:
        output[k] = left[i]
        i += 1
        k += 1

    while j < n_right:
        output[k] = right[j]
        j += 1
        k += 1

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])

    merged = np.empty_like(arr)
    merge(left, right, merged)
    return merged


def cuda_merge_sort(arr):
    start_time = time.time()
    if len(arr) <= 1:
        # end_time = time.time()
        return arr
    mid = len(arr) // 2 
    left = merge_sort(arr[:mid]) 
    right = merge_sort(arr[mid:])

    merged = np.empty_like(arr)
    encoded_merge_gpu = cuda.mem_alloc(merged.nbytes)
    cuda.memcpy_htod(encoded_merge_gpu, merged)
    merge(left, right, merged)
    end_time = time.time()
    return merged, end_time - start_time

# Example usage
#input_array = [8, 4, 2, 9, 3, 6, 1, 7, 5, 10, 567, 487, 5893, 930, 740, 36, 24]
# print("Input array:", input_array)

# Using the regular merge sort
start = time.time()
input_array = [random.randint(1, 1000000) for _ in range(100000)]
# print(input_list)
sorted_array = merge_sort(input_array)
end = time.time()
print("Sorted array (Merge Sort):", sorted_array)
print("Time taken on CPU:", end - start)

# Using the CUDA merge sort
cuda_sorted_array, time_taken_gpu = cuda_merge_sort(input_array)
print("Sorted array (CUDA Merge Sort):", cuda_sorted_array)
print("Time taken on GPU:", time_taken_gpu) 

Sorted array (Merge Sort): [    20     32     47 ... 999969 999995 999997]
Time taken on CPU: 0.9441301822662354
Sorted array (CUDA Merge Sort): [    20     32     47 ... 999969 999995 999997]
Time taken on GPU: 0.8938086032867432
